In [ ]:
# ---------------------------------------------
# Retail Sales Forecasting – Time Series Analysis
# ---------------------------------------------

# Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# ---------------------------------------------
# Step 1: Load and Prepare Dataset
# ---------------------------------------------
df = pd.read_csv("salesdaily.csv")  # Replace with your file path

# Convert date column
df['datum'] = pd.to_datetime(df['datum'])

# Create Total Sales column by summing all product categories
df['Total_Sales'] = df[['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']].sum(axis=1)

# Sort by date
df = df.sort_values('datum')

# ---------------------------------------------
# Step 2: Visualize Sales Trend
# ---------------------------------------------
plt.figure(figsize=(10, 5))
plt.plot(df['datum'], df['Total_Sales'], color='orange')
plt.title("Retail Daily Total Sales Trend")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.show()

# ---------------------------------------------
# Step 3: ARIMA Model Forecasting
# ---------------------------------------------
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# Fit ARIMA model
model = ARIMA(train['Total_Sales'], order=(5, 1, 0))
model_fit = model.fit()

# Forecast future values
forecast = model_fit.forecast(steps=len(test))
forecast = np.maximum(forecast, 0)

# Evaluate performance
mae = mean_absolute_error(test['Total_Sales'], forecast)
rmse = np.sqrt(mean_squared_error(test['Total_Sales'], forecast))
print(f"ARIMA Model - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Plot ARIMA Forecast
plt.figure(figsize=(10, 5))
plt.plot(train['datum'], train['Total_Sales'], label='Train')
plt.plot(test['datum'], test['Total_Sales'], label='Test', color='orange')
plt.plot(test['datum'], forecast, label='ARIMA Forecast', color='green')
plt.title("ARIMA Forecast vs Actual Sales")
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend()
plt.show()

# ---------------------------------------------
# Step 4: Prophet Model Forecasting
# ---------------------------------------------
# Prepare data for Prophet
prophet_df = df[['datum', 'Total_Sales']].rename(columns={'datum': 'ds', 'Total_Sales': 'y'})

# Fit model
model_prophet = Prophet(daily_seasonality=True)
model_prophet.fit(prophet_df)

# Create future dataframe (next 180 days)
future = model_prophet.make_future_dataframe(periods=180)
forecast_prophet = model_prophet.predict(future)

# Plot forecast
model_prophet.plot(forecast_prophet)
plt.title("Prophet Forecast for Sales")
plt.show()

# ---------------------------------------------
# Step 5: Export for Power BI
# ---------------------------------------------
# Save actual and forecasted values for Power BI
forecast_export = forecast_prophet[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecast_export.columns = ['Date', 'Predicted_Sales', 'Lower_Bound', 'Upper_Bound']

# Merge with actual data
final_data = pd.merge(df[['datum', 'Total_Sales']], forecast_export,
                      left_on='datum', right_on='Date', how='left')

final_data.to_excel("Retail_Sales_Forecast.xlsx", index=False)
print("✅ Forecast results exported successfully to 'Retail_Sales_Forecast.xlsx'")

# ---------------------------------------------
# Step 6: Optional Automation Example
# ---------------------------------------------
# Example: Clean and prepare automatically for re-run
def automated_data_cleaning(file_path):
    data = pd.read_csv(file_path)
    data['datum'] = pd.to_datetime(data['datum'])
    data['Total_Sales'] = data[['M01AB', 'M01AE', 'N02BA', 'N02BE', 'N05B', 'N05C', 'R03', 'R06']].sum(axis=1)
    return data.sort_values('datum')

# Example usage:
# df_cleaned = automated_data_cleaning("salesdaily.csv")